### Import all basic libraries

In [45]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


### Import Dataset

In [46]:
path = "dataset.csv"

df = pd.read_csv(path)
if (df.empty):
    print("Dataset not loaded")
else:
    print("Dataset loaded successfully")


print("Path to dataset files:", path)

df.head()

Dataset loaded successfully
Path to dataset files: dataset.csv


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Clean Data

In [47]:
# Check for missing values
df.isnull().sum()  # No missing values

# Drop customerID column
df.drop(columns=["customerID"], inplace=True)

# Convert TotalCharges to float
df["TotalCharges"] = df["TotalCharges"].replace(" ", "0")  # Replace empty strings
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")  # Convert to float

# Convert binary categorical columns to 1/0
df["gender"] = df["gender"].map({"Male": 1, "Female": 0})
df["Partner"] = df["Partner"].map({"Yes": 1, "No": 0})
df["Dependents"] = df["Dependents"].map({"Yes": 1, "No": 0})
df["PhoneService"] = df["PhoneService"].map({"Yes": 1, "No": 0})
df["MultipleLines"] = df["MultipleLines"].map({"Yes": 1, "No": 0, "No phone service": 0})

# Convert "No internet service" to "No" for consistency
df["OnlineSecurity"] = df["OnlineSecurity"].replace({"No internet service": "No"})
df["OnlineBackup"] = df["OnlineBackup"].replace({"No internet service": "No"})
df["DeviceProtection"] = df["DeviceProtection"].replace({"No internet service": "No"})
df["TechSupport"] = df["TechSupport"].replace({"No internet service": "No"})
df["StreamingTV"] = df["StreamingTV"].replace({"No internet service": "No"})
df["StreamingMovies"] = df["StreamingMovies"].replace({"No internet service": "No"})

# Convert remaining binary columns to 1/0
df["OnlineSecurity"] = df["OnlineSecurity"].map({"Yes": 1, "No": 0})
df["OnlineBackup"] = df["OnlineBackup"].map({"Yes": 1, "No": 0})
df["DeviceProtection"] = df["DeviceProtection"].map({"Yes": 1, "No": 0})
df["TechSupport"] = df["TechSupport"].map({"Yes": 1, "No": 0})
df["StreamingTV"] = df["StreamingTV"].map({"Yes": 1, "No": 0})
df["StreamingMovies"] = df["StreamingMovies"].map({"Yes": 1, "No": 0})


# One-Hot Encode categorical columns (drop first to avoid redundancy) --> e.g if we have a column for "Fibre Optic", "DSL", "No", we can drop the "No" column because if the other two are 0, then it's "No"
df = pd.get_dummies(df, columns=["InternetService", "Contract", "PaymentMethod"], drop_first=True)

# Convert PaperlessBilling and Churn to binary
df["PaperlessBilling"] = df["PaperlessBilling"].map({"Yes": 1, "No": 0})
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})

# Final check
print(df.info())  # Verify all columns are numerical
df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   gender                                 7043 non-null   int64  
 1   SeniorCitizen                          7043 non-null   int64  
 2   Partner                                7043 non-null   int64  
 3   Dependents                             7043 non-null   int64  
 4   tenure                                 7043 non-null   int64  
 5   PhoneService                           7043 non-null   int64  
 6   MultipleLines                          7043 non-null   int64  
 7   OnlineSecurity                         7043 non-null   int64  
 8   OnlineBackup                           7043 non-null   int64  
 9   DeviceProtection                       7043 non-null   int64  
 10  TechSupport                            7043 non-null   int64  
 11  Stre

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,MonthlyCharges,TotalCharges,Churn,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,0,1,0,0,0,1,0,...,29.85,29.85,0,0,0,0,0,0,1,0
1,1,0,0,0,34,1,0,1,0,1,...,56.95,1889.50,0,0,0,1,0,0,0,1
2,1,0,0,0,2,1,0,1,1,0,...,53.85,108.15,1,0,0,0,0,0,0,1
3,1,0,0,0,45,0,0,1,0,1,...,42.30,1840.75,0,0,0,1,0,0,0,0
4,0,0,0,0,2,1,0,0,0,0,...,70.70,151.65,1,1,0,0,0,0,1,0


### Split Data

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import GridSearchCV

# Create feature matrix
X = df.drop(columns=["Churn"])

# Create target vector 
y = df["Churn"]

forest = RandomForestClassifier(random_state=0, max_depth=5)

# Test, train and split --> 90% training data and 10% testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=0)

# Further split the training data into a validation set and training set --> 20% is validation set and 80% is training set
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

"""
We create a validation set to help us adjust and fine tune our parameters
"""


# Fit the model on the training data (training final dataset)
forest.fit(X_train_final, y_train_final)

print("Accuracy on training set: {:.3f}".format(forest.score(X_train_final, y_train_final)))
print("Accuracy on validation set: {:.3f}".format(forest.score(X_val, y_val)))

# Use our current model to predict based on our X_val training data
y_val_pred = forest.predict(X_val)
# Compare to actual validation set to get precision and recall scores
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)

print("Precision:", precision)
print("Recall:", recall)

"""
Initial results:

Accuracy on training set: 0.807
Accuracy on validation set: 0.781
Precision: 0.7236180904522613
Recall: 0.3923705722070845

Utilize GridSearchCV to find the best mix of parameters
"""

# Define param grid
param_grid = {
    "max_depth": [5,10,15,20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1,2,4]
}

# Perform grid search
grid_search = GridSearchCV(RandomForestClassifier(random_state=0), param_grid, cv=5, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train_final, y_train_final)

# Find the best parameters
print("Best parameters:", grid_search.best_params_)


Accuracy on training set: 0.807
Accuracy on validation set: 0.781
Precision: 0.7236180904522613
Recall: 0.3923705722070845
Best parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}


### Creating the best forest model

In [ ]:
forest_best = RandomForestClassifier(random_state=0, max_depth=10, min_samples_leaf=2, min_samples_split=10)
forest_best.fit(X_train_final, y_train_final)

print("Accuracy on training set: {:.3f}".format(forest_best.score(X_train_final, y_train_final)))
print("Accuracy on validation set: {:.3f}".format(forest_best.score(X_val, y_val)))

y_val_pred = forest_best.predict(X_val)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)

print("Precision:", precision)
print("Recall:", recall)

""" We have traded off precision for a higher amount of recall
because we want to catch as many people cancelling
as possible to better improve our service"""



Accuracy on training set: 0.858
Accuracy on validation set: 0.795
Precision: 0.7034220532319392
Recall: 0.5040871934604905
